# Проектная работа: Защита персональных данных клиентов

### Описание проекта

Нам нужно защитить данные клиентов страховой компании «Хоть потоп». Для этого разработаем такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Также обоснуем корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

**Целью** данной работы является разработка алгоритма преобразования данных. **Задачами** проектной работы являются:
1. Изучить вопрос изменения качества линейной регрессии, если признаки умножают на обратимую матрицу;
2. Предложиить алгоритм преобразования данных для решения задачи;
3. Реализовать предложенный алгоритм.

**Описание данных**:
1. **Признаки**: пол, возраст и зарплата застрахованного, количество членов его семьи.
2. **Целевой признак**: количество страховых выплат клиенту за последние 5 лет.

**План проектной работы**:
1. Загрузка и изучение данных;
2. Изучение вопроса о том, как меняется качество линейной регрессии;
3. Алгоритм преобразования данных;
4. Реализация алгоритма с применением матричных операций;
5. Выводы.

### Шаг 1. Загрузка и изучение данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

In [2]:
df = pd.read_csv("insurance.csv")
mapping = {"Возраст" : "{:,.0f}", "Зарплата" : "{:,.2f}"}
df.head().style.format(mapping)

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41,"49,600.00",1,0
1,0,46,"38,000.00",1,1
2,0,29,"21,000.00",0,0
3,0,21,"41,700.00",2,0
4,1,28,"26,100.00",0,0


In [3]:
df.shape

(5000, 5)

Представленный датасет содержит 5 тыс. строк и 5 столбцов.

In [4]:
df.dtypes

Пол                    int64
Возраст              float64
Зарплата             float64
Члены семьи            int64
Страховые выплаты      int64
dtype: object

Типы данных в столбцах корректные, за исключением `Возраст`. Поменяем тип данных в этом столбце на `int`. 

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


Датасет не содержит пропущенных значений.

Проверим, что в каждом столбце представлены адекватные данные, то есть не содержат такие значения, которые логически не подходят под данный столбец.

In [6]:
for col in ["Пол", "Члены семьи", "Страховые выплаты"]:
    print(f"{col}:" , df[col].unique())
    print()

Пол: [1 0]

Члены семьи: [1 0 2 4 3 5 6]

Страховые выплаты: [0 1 2 3 5 4]



In [7]:
for col in ["Зарплата", "Возраст"]:
    print(f"Содержится ли в столбце {col} нулевой или отрицательное значение:", any(df[col].unique() <= 0))
    print()

Содержится ли в столбце Зарплата нулевой или отрицательное значение: False

Содержится ли в столбце Возраст нулевой или отрицательное значение: False



Поменяем тип данных в столбце `Возраст`, так как возраст указывается в целых числах.

In [8]:
df["Возраст"] = df["Возраст"].astype("int")
print(df["Возраст"].dtypes)

int64


### Выводы
---
Таким образом, качество представленных данных оценивается как высокое. В датасете содержится 5 тыс. строк и 5 столбцов. Пропущенные значения отсутствуют. Дополнительно мы проверили, что в столбцах не содержатся неадекватные значения (нулевые или отрицательные), и поменяли тип данных в столбце `Возраст`.

### Шаг 2. Изучение вопроса о том, как меняется качество линейной регрессии

Обозначения для работы с уровнениями линейной регрессии:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — обратимая матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

**Предсказания**:

$$
a = Xw
$$

**Задача обучения**:

$$
w = \arg\min_w MSE(Xw, y)
$$

**Вектор весов линейной регрессии**:

$$
w = (X^T X)^{-1} X^T y
$$

Подставим в вектор предсказаний $a$ выражение в правой части из вектора весов линейной регрессии $w$:

$$
a = X(X^T X)^{-1} X^T y
$$

Пусть у нас есть обратимая матрица $P$, мы домножаем на нее матрицу признаков $X$ и получаем:

$$
a' = XP((XP)^T XP)^{-1} (XP)^T y
$$

Необходимо **доказать**, что:

$$
a = a'
$$

**Доказательство**:

Раскроем скобки в $a'$, используя:

1. Свойство операции транспонирования матриц:

$$
(AB)^{T} = B^{T} A^{T}
$$

2. Свойство обратной матрицы:

$$
(AB)^{-1} = B^{-1} A^{-1}
$$

3. Определение обратной матрицы:

$$
AA^{-1} = A^{-1}A = E
$$


Таким образом, получаем следующее:
1. Раскроем $(XP)^T$ в середине выражения и возьмем в скобки $(X^T X)$:

$$ a' = XP(P^T (X^T X) P)^{-1} (XP)^T y $$

2. Раскроем $(XP)^T$ в конце выражения:

$$ a' = XP(P^T (X^T X) P)^{-1} P^T X^T y $$

3. Раскроем обращение матрицы, состоящей из произведения матриц $(P^T (X^T X) P)^{-1}$, и будем считать, что матрицы $(P^T (X^T X)$ и $P$ - отдельные переменные:

$$ a' = XP P^{-1}(P^T (X^T X)^{-1} P^T X^T y $$

4. Снова раскроем обращение матрицы, состоящей из произведения матриц $(P^T (X^T X)^{-1}$:

$$ a' = XP P^{-1}(X^T X)^{-1}(P^T)^{-1}P^T X^T y $$

5. Воспользуемся определением обратной матрицы, то есть умножение матрицы на обратную матрицу дает единичную матрицу, и в итоге получаем:

$$ a' = X(X^T X)^{-1}X^Ty $$

### Шаг 3. Алгоритм преобразования данных

Таким образом, как было доказано в предыдущем шаге, умножение матрицы признаков на обратимую матрицу не должно привести к изменению вектора предсказаний. Поэтому **алгоритм** может быть предложен, исходя из этого доказательства, и будет реализован в виде функции:

1. Функция на вход принимает матрицу признаков и количество столбцов в матрице признаков ($n$).

2. Исходя из этого количества, генерируется случайная квадратная матрица размера $nxn$. При этом значения в матрице случайным образом генерируется из выборки от 1 до 10.

3. Главным условием для новой матрицы является ее обратимость. Для проверки данного условия в функции будет считаться определитель матрицы. Если определитель равен нулю, то генерируется новая матрица.

4. Функция возвращает измененную матрицу признаков и вновь сгенерированную матрицу.

In [9]:
def matrix_generation(features, number_of_columns):
    np.random.seed(12345)
    new_matrix = np.random.randint(1, 10, size = (number_of_columns, number_of_columns))
    new_matrix_determinant = np.linalg.det(new_matrix)
    if new_matrix_determinant == 0:
        while new_matrix_determinant == 0:
            np.random.seed(12345)
            new_matrix = np.random.randint(1, 10, size = (number_of_columns, number_of_columns))
            new_matrix_determinant = np.linalg.det(new_matrix)
    return features.dot(new_matrix), new_matrix     

Пусть у нас есть матрица признаков $X$ размерности $5x3$. Создадим для нее матрицу $P$, которая должна маскировать данные клиентов.

In [10]:
np.random.seed(12345)
X = np.random.randint(1, 10, size = (5, 3))
X

array([[3, 6, 2],
       [5, 6, 3],
       [2, 7, 2],
       [8, 7, 1],
       [3, 2, 3]])

In [11]:
X.shape

(5, 3)

In [12]:
XP, P = matrix_generation(X, X.shape[1])
display(XP)
print()
display(P)

array([[43, 68, 28],
       [51, 87, 34],
       [45, 68, 29],
       [61, 97, 39],
       [25, 51, 18]])

array([[3, 6, 2],
       [5, 6, 3],
       [2, 7, 2]])

In [13]:
XP.shape

(5, 3)

In [14]:
np.linalg.det(P)

-5.000000000000003

### Шаг 4. Реализация алгоритма с применением матричных операций

Разделим данные на обучающую и тестовую:

In [15]:
features = df.drop('Страховые выплаты', axis = 1)
target = df['Страховые выплаты']

In [16]:
features_train, features_test, target_train, target_test = train_test_split(features, 
                                                            target, test_size = 0.25, random_state = 12345)

print("Размерность обучающей выборки для признаков:", features_train.shape)
print("Размерность обучающей выборки для таргетов:", target_train.shape)
print("Размерность тестовой выборки для признаков:", features_test.shape)
print("Размерность тестовой выборки для таргетов:", target_test.shape)

Размерность обучающей выборки для признаков: (3750, 4)
Размерность обучающей выборки для таргетов: (3750,)
Размерность тестовой выборки для признаков: (1250, 4)
Размерность тестовой выборки для таргетов: (1250,)


Напишем функцию для оценки модели линейной регрессии:

In [17]:
def model_evaluation(features_train, features_test, target_train, target_test):
    model = LinearRegression()
    model.fit(features_train, target_train)
    predictions = model.predict(features_test)

    r2 = r2_score(target_test, predictions)

    print("Коэффициент детерминации:", "{:,.2%}".format(r2))

Оценим модель линейной регрессии на исходных данных:

In [18]:
model_evaluation(features_train, features_test, target_train, target_test)

Коэффициент детерминации: 43.52%


Посмотрим, как стандартизация признаков в исходных данных повлияет на коэффициент детерминации:

In [19]:
def data_scale(features_train, features_test):
    scaler = StandardScaler()
    scaler.fit(features_train)
    features_train_scaled = scaler.transform(features_train)
    features_test_scaled = scaler.transform(features_test)
    return features_train_scaled, features_test_scaled

In [20]:
features_train_scaled, features_test_scaled = data_scale(features_train, features_test)

In [21]:
model_evaluation(features_train_scaled, features_test_scaled, target_train, target_test)

Коэффициент детерминации: 43.52%


В результате стандартизации набора данных коэффициент детерминации не изменился.

Теперь оценим аналогичные модели на новых данных:

In [22]:
new_features, P = matrix_generation(features, features.shape[1])

Сравним исходные данные с новым набором признаков, также посмотрим на сгенерированную матрицу $P$:

In [23]:
display(features.head())
print()
display(new_features.head())
print()
print(P)

,Пол,Возраст,Зарплата,Члены семьи
0,1,41,49600.0,1
1,0,46,38000.0,1
2,0,29,21000.0,0
3,0,21,41700.0,2
4,1,28,26100.0,0


,0,1,2,3
0,99452.0,396931.0,347287.0,49899.0
1,76279.0,304140.0,266095.0,38329.0
2,42174.0,168087.0,147058.0,21203.0
3,83532.0,333667.0,291948.0,41861.0
4,52371.0,208890.0,182758.0,26301.0



[[3 6 2 5]
 [6 3 2 7]
 [2 8 7 1]
 [3 2 3 7]]


Проверим, что функция сработала правильно, посчитав определитель матрицы $P$:

In [24]:
np.linalg.det(P)

-616.9999999999995

In [25]:
P.shape

(4, 4)

Определитель матрицы $P$ отличается от нуля, а сама матрица является квадратной. Снова подели данные на обучающую и тестовую и оценим модели:

In [26]:
new_features_train, new_features_test, new_target_train, new_target_test = train_test_split(new_features, 
                                                            target, test_size = 0.25, random_state = 12345)

print("Размерность новой обучающей выборки для признаков:", new_features_train.shape)
print("Размерность новой обучающей выборки для таргетов:", new_target_train.shape)
print("Размерность новой тестовой выборки для признаков:", new_features_test.shape)
print("Размерность новой тестовой выборки для таргетов:", new_target_test.shape)

Размерность новой обучающей выборки для признаков: (3750, 4)
Размерность новой обучающей выборки для таргетов: (3750,)
Размерность новой тестовой выборки для признаков: (1250, 4)
Размерность новой тестовой выборки для таргетов: (1250,)


In [27]:
model_evaluation(new_features_train, new_features_test, new_target_train, new_target_test)

Коэффициент детерминации: 43.52%


Таким образом, качество линейной регрессии не отличается до и после преобразования. Посмотрим теперь на стандартизированных данных:

In [28]:
new_features_train_scaled, new_features_test_scaled = data_scale(new_features_train, new_features_test)

In [29]:
model_evaluation(new_features_train_scaled, new_features_test_scaled, new_target_train, new_target_test)

Коэффициент детерминации: 43.52%


Как мы видим, стандартизация набора данных не приводит к изменению качества линейной регрессии. 

### Шаг 5. Выводы
---
В данной работе:

1. Аналитически было доказано, что преобразование матрицы признаков не приводит к изменению вектора предсказаний модели.
2. Был предложен алгоритм преобразования данных.
3. Были оценены модели до и после преобразования данных.

Таким образом, алгоритм преобразования данных разработан, хотя и предложенный вариант является достаточно тривиальным. Тем не менее, преобразование данных не привело к изменению качества линейной регрессии. Значения коэффициента детерминации до и после преобразования остались неизменными. Стандартизация признаков так же не приводит к ухудшению $R^2$. 